In [21]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
from load_data import Balance, Instruments, AdvisedPortfolios, PriceDB, Singleton
from utils import get_current_port, get_advised_port, get_recommendation

from asset import Asset
from price import Price
from portfolio import Portfolio

import pandas as pd

In [19]:
userid='A01'
risk_profile=2

#### `userid`로 사용자 최근잔고 잔고 불러오기
- `balance`
- `advised_pf`

In [23]:
Balance.instance().data

FileNotFoundError: [Errno 2] No such file or directory: './data/processed/balance_s.pkl'

In [10]:
balance = get_current_port(userid=userid)

FileNotFoundError: [Errno 2] No such file or directory: './data/processed/balance_s.pkl'

In [4]:
__name__

'__main__'

In [5]:
new_port = get_advised_port(risk_profile=risk_profile)

FileNotFoundError: [Errno 2] No such file or directory: './data/processed/advised_portfolios.pkl'

In [6]:
!pwd

/Users/msyeom/iCloud Drive (Archive)/Documents/Finance/Work/kisra/src/models


In [24]:
advised_pf = pd.read_pickle('../../data/processed/advised_portfolios.pkl')

In [26]:
df = advised_pf.loc[(advised_pf.risk_profile==2) & (advised_pf.date=='2021-01-04')]

In [27]:
df

,date,risk_profile,itemcode,weights,tracking_code,itemname,price,volume,trading_amt_mln,asset_class
1820,2021-01-04,2,A122260,0.25,BOK_D1,KOSEF 통안채1년,101015.0,2773.0,53191.510933,Fixed Income
1821,2021-01-04,2,A114260,0.25,KTB_D3,KODEX 국고채3년,58100.0,4571.0,547.479291,Fixed Income
1822,2021-01-04,2,A130730,0.25,MM,KOSEF 단기자금,100835.0,88085.0,59165.272681,Fixed Income
1823,2021-01-04,2,A329650,0.15,BAL_WO3KTB7,KODEX TRF3070,10735.0,33117.0,122.211411,Alternative
1824,2021-01-04,2,A266370,0.10,IT,KODEX IT,20805.0,152199.0,539.058393,Equity


In [5]:
old_new = pd.merge(balance.loc[:, ['itemcode', 'quantity', 'price', 'value', 'wt']], new_port.loc[:, ['itemcode', 'wt']],
                left_on=['itemcode'], right_on=['itemcode'], how='outer', suffixes=['_old', '_new'])

In [6]:
old_new

,itemcode,quantity,price,value,wt_old,wt_new
0,C000001,19565.0,1.00,19565.00,0.027733,NaN
1,D000001,350000.0,1.01,353500.00,0.501074,NaN
2,A069500,1.0,42936.90,42936.90,0.060862,NaN
3,A360200,2.0,10430.40,20860.80,0.029569,NaN
4,A196230,1.0,108498.00,108498.00,0.153792,NaN
5,A302190,1.0,51841.35,51841.35,0.073483,NaN
6,A329750,8.0,9645.00,77160.00,0.109372,NaN
7,A367380,3.0,10374.00,31122.00,0.044114,NaN
8,A122260,NaN,NaN,NaN,NaN,0.250000
9,A130730,NaN,NaN,NaN,NaN,0.250000


In [7]:
# old_new.loc[old_new.itemcode_old.isna(), 'itemcode_old'] = old_new.loc[old_new.itemcode_old.isna(), 'itemcode_new']
# old_new.loc[old_new.itemcode_new.isna(), 'itemcode_new'] = old_new.loc[old_new.itemcode_new.isna(), 'itemcode_old']
old_new.fillna(value=0, inplace=True)

In [8]:
old_new

,itemcode,quantity,price,value,wt_old,wt_new
0,C000001,19565.0,1.00,19565.00,0.027733,0.000000
1,D000001,350000.0,1.01,353500.00,0.501074,0.000000
2,A069500,1.0,42936.90,42936.90,0.060862,0.000000
3,A360200,2.0,10430.40,20860.80,0.029569,0.000000
4,A196230,1.0,108498.00,108498.00,0.153792,0.000000
5,A302190,1.0,51841.35,51841.35,0.073483,0.000000
6,A329750,8.0,9645.00,77160.00,0.109372,0.000000
7,A367380,3.0,10374.00,31122.00,0.044114,0.000000
8,A122260,0.0,0.00,0.00,0.000000,0.250000
9,A130730,0.0,0.00,0.00,0.000000,0.250000


In [9]:
assets = old_new.loc[(old_new.itemcode != 'C000001') & (old_new.itemcode != 'DEPOSIT'), :]
cash = old_new.loc[(old_new.itemcode=='CASH') | (old_new.itemcode=='DEPOSIT'), :]
old_assets = assets.drop(['wt_new'], axis=1)
old_cash = cash.drop(['wt_new'], axis=1)
old_tickers = assets.itemcode.tolist()
old_quantities = assets.quantity.astype(int).tolist()
old_prices = assets.price.tolist()
cash_amounts = cash.value.tolist()
cash_currency = ['KRW']*len(cash_amounts)

In [10]:
# old_assets = balance.loc[(balance.tracking_code != 'CASH') & (balance.tracking_code != 'DEPOSIT'), :]
# old_cash = balance.loc[(balance.tracking_code=='CASH') | (balance.tracking_code=='DEPOSIT'), :]
# old_tickers = old_assets.itemcode.tolist()
# old_quantities = old_assets.quantity.tolist()
# old_prices = old_assets.price.tolist()
# cash_amounts = old_cash.value.tolist()
# cash_currency = ['KRW']*len(cash_amounts)

In [11]:
p = Portfolio()
p.easy_add_assets(tickers=old_tickers, quantities=old_quantities, prices=old_prices)
p.easy_add_cash(amounts=cash_amounts, currencies=cash_currency)
p.selling_allowed = True

In [12]:
p.assets

{'D000001': <asset.Asset at 0x7fe984a6f7c0>,
 'A069500': <asset.Asset at 0x7fe984a6f880>,
 'A360200': <asset.Asset at 0x7fe984a6f940>,
 'A196230': <asset.Asset at 0x7fe984a6fa60>,
 'A302190': <asset.Asset at 0x7fe984a6fb50>,
 'A329750': <asset.Asset at 0x7fe984a6fc10>,
 'A367380': <asset.Asset at 0x7fe984a6fca0>,
 'A122260': <asset.Asset at 0x7fe984a6fd30>,
 'A130730': <asset.Asset at 0x7fe984a6fdc0>,
 'A153130': <asset.Asset at 0x7fe984a6fe80>,
 'A114260': <asset.Asset at 0x7fe984a6ff70>,
 'A214980': <asset.Asset at 0x7fe9846867f0>,
 'A102110': <asset.Asset at 0x7fe984686be0>}

In [13]:
old_new.loc[(old_new.itemcode != 'CASH') & (old_new.itemcode != 'DEPOSIT'), 'itemcode']

0     C000001
1     D000001
2     A069500
3     A360200
4     A196230
5     A302190
6     A329750
7     A367380
8     A122260
9     A130730
10    A153130
11    A114260
12    A214980
13    A102110
Name: itemcode, dtype: object

In [14]:
new_tickers = old_new.loc[(old_new.itemcode != 'CASH') & (old_new.itemcode != 'DEPOSIT'), 'itemcode'].tolist()
new_wt = (old_new.loc[(old_new.itemcode != 'CASH') & (old_new.itemcode != 'DEPOSIT'), 'wt_new']*100).tolist()     # 단위가 %이므로 100을 곱한다.

In [15]:
target_asset_alloc = dict(zip(new_tickers, new_wt))    

In [16]:
p.rebalance(target_asset_alloc, verbose=True)

/Users/msyeom/iCloud Drive (Archive)/Documents/Finance/Work/kisra/src/models/rebalancing_helper.py:46: RuntimeWarning: divide by zero encountered in double_scalars
  (sol_mv - portfolio.assets[ticker].market_value_in(


OverflowError: cannot convert float infinity to integer

In [16]:
target_asset_alloc['A069500']

KeyError: 'A069500'

In [13]:
p.assets

{'A069500': <asset.Asset at 0x1e0cead8b20>,
 'A360200': <asset.Asset at 0x1e0cead8be0>,
 'A196230': <asset.Asset at 0x1e0cead8cd0>,
 'A302190': <asset.Asset at 0x1e0cead8d90>,
 'A329750': <asset.Asset at 0x1e0cead8e80>,
 'A367380': <asset.Asset at 0x1e0cead8f40>}

In [63]:
new_port.wt.sum()

0.9999999998925693

In [68]:
import numpy as np

In [70]:
# order target_allocation dict in the same order as assets dict and upper key
target_allocation_reordered = {}
try:
    for key in assets:
        target_allocation_reordered[key] = target_allocation[key]
except:
    raise Exception(
        "'target_allocation not compatible with the assets of the portfolio."
    )

target_allocation_np = np.fromiter(
    target_allocation_reordered.values(), dtype=float)

Exception: 'target_allocation not compatible with the assets of the portfolio.

In [73]:
p.assets

{}

In [19]:
price_db = PriceDB.instance().data

In [21]:
set(price_db.itemtype)

{'ETF'}